In [1]:
import cplex
import pandas as pd
from sklearn.linear_model import LogisticRegression
from recourse.paths import *
from recourse.builder import RecourseBuilder, _SOLVER_TYPE_CBC, _SOLVER_TYPE_CPX
from recourse.action_set import ActionSet
import numpy as np

data_name = 'german'
data_file = test_dir / ('%s_processed.csv' % data_name)

## load dataset
data_df = pd.read_csv(data_file)
outcome_name = data_df.columns[0]
y = data_df[outcome_name]
X = data_df.drop([outcome_name, 'Gender', 'PurposeOfLoan', 'OtherLoansAtStore'], axis=1)

# setup actionset
action_set = ActionSet(X = X)
immutable_attributes = [
                        'Age', 
                        'Single', 
                        'JobClassIsSkilled',
                        'ForeignWorker', 
                        'OwnsHouse', 
                        'RentsHouse'
                       ]
action_set[immutable_attributes].mutable = False
action_set['CriticalAccountOrLoansElsewhere'].step_direction = -1
action_set['CheckingAccountBalance_geq_0'].step_direction = 1

# fit classifier, get median score, and get denied individuals.
clf = LogisticRegression(max_iter=1000, solver = 'lbfgs')
clf.fit(X, y)
coefficients = clf.coef_[0]
intercept = clf.intercept_[0]
scores = pd.Series(clf.predict_proba(X)[:, 1])
p = scores.median()
denied_individuals = scores.loc[lambda s: s <= p].index

idx = denied_individuals[0]
x = X.values[idx]

## CPLEX
fb_cplex = RecourseBuilder(
    solver=_SOLVER_TYPE_CPX,
    coefficients=coefficients,
    intercept=intercept - (np.log(p / (1. - p))),
    action_set=action_set,
    x=x
)
cplex_solution = fb_cplex.fit()

## CBC
fb_cbc = RecourseBuilder(
    solver=_SOLVER_TYPE_CBC,
    coefficients=coefficients,
    intercept=intercept - (np.log(p / (1. - p))),
    action_set=action_set,
    x=x
)
cbc_solution = fb_cbc.fit()

In [2]:
from recourse.auditor import RecourseAuditor

In [3]:
auditor_cpx = RecourseAuditor(clf = clf, action_set = action_set, solver= _SOLVER_TYPE_CPX)

In [4]:
cpx_audit = auditor_cpx.audit(X.iloc[:50])

In [5]:
auditor_cbc = RecourseAuditor(clf = clf, action_set = action_set, solver= _SOLVER_TYPE_CBC)

In [6]:
cbc_audit = auditor_cbc.audit(X.iloc[:50])

In [11]:
np.isclose(cbc_audit['cost'].fillna(0), cpx_audit['cost'].fillna(0))

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [54]:
p = .5

In [24]:
idx = 1
x = X.values[idx]

## CPLEX
fb_cplex = RecourseBuilder(
    solver=_SOLVER_TYPE_CPX,
    coefficients=coefficients,
    intercept=intercept,
    action_set=action_set,
    x=x
)
cplex_solution = fb_cplex.fit()

## CBC
fb_cbc = RecourseBuilder(
    solver=_SOLVER_TYPE_CBC,
    coefficients=coefficients,
    intercept=intercept,
    action_set=action_set,
    x=x
)
cbc_solution = fb_cbc.fit()

In [144]:
cplex_solution

{'cost': 0.05199998545391072,
 'feasible': True,
 'status': 'integer optimal solution',
 'costs': array([0.        , 0.        , 0.        , 0.01506283, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.05199999,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]),
 'actions': array([ 0.,  0.,  0., -2.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
 'upperbound': 0.05200500550711558,
 'lowerbound': 0.05200500550711558,
 'gap': 0.0,
 'iterations': 40,
 'nodes_processed': 0,
 'nodes_remaining': 0,
 'runtime': 0.03125}

In [145]:
cbc_solution

{'cost': 0.051999985,
 'feasible': True,
 'status': 'optimal',
 'costs': array([0.        , 0.        , 0.        , 0.        , 0.0002685 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.05199999,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]),
 'actions': array([ 0.,  0.,  0.,  0., -6.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
 'upperbound': inf,
 'lowerbound': inf,
 'gap': inf,
 'iterations': 0,
 'nodes_processed': 0,
 'nodes_remaining': 0,
 'runtime': 0.015625}

In [123]:
mip = fb_cbc._mip
sol = {j: {'a': mip.a[j], 'u': mip.u[j](), 'c': mip.c[j]} for j in mip.JK}
df = (pd.DataFrame.from_dict(sol, orient = "index").loc[lambda df: df['u'] == 1])

In [79]:
# pd.DataFrame.from_dict(sol, orient = "index").loc[lambda df: df['u'] == 1]

In [139]:
(x + cplex_solution['actions']).dot(coefficients ) + intercept

0.6964119402445514

In [140]:
intercept

0.7946385917974634

In [124]:
fb_cbc.score()

0.6960454355973127

In [125]:
fb_cplex.score()

0.6960454355973127

In [126]:
fb_cbc.intercept

0.7946385917974634

In [127]:
fb_cplex.intercept

0.7946385917974634

In [146]:
fb_cplex.prediction()

-1.0

In [147]:
fb_cplex.score()

-0.5713418790894296

In [100]:
t = fb_cbc._get_mip_build_info()

In [118]:
cplex_solution['actions']

array([ 0.,  0.,  0.,  0., -5.,  0.,  0.,  0.,  0., -0.,  0., -0., -0.,
       -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [113]:
pd.Series(clf.predict_proba(X)[:, 1]).loc[lambda s: s<.5].head()

1     0.360927
5     0.471771
10    0.492662
11    0.299070
17    0.343244
dtype: float64

In [107]:
coefficients.shape

(26,)

In [149]:
intercept

0.7946385917974634

In [115]:
(X.dot(coefficients) + intercept).loc[lambda s: s < 0].head()

1    -0.571342
5    -0.113035
10   -0.029354
11   -0.851732
17   -0.648870
dtype: float64

In [116]:
pd.Series(clf.predict_proba(X)[:, 1]).loc[lambda s: s<.5].index == (X.dot(coefficients) + intercept).loc[lambda s: s < 0].index

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [15]:
denied_individuals = scores.loc[lambda s: s <= .5].index
idx = denied_individuals[1]
x = X.iloc[idx].values

In [16]:
fb_cbc = RecourseBuilder(
    solver=_SOLVER_TYPE_CBC,
    coefficients=coefficients,
    intercept=intercept,
    action_set=action_set,
    x=x
)
cbc_solution = fb_cbc.fit()

In [3]:
from recourse.flipset import Flipset

In [ ]:
items_cpx  = Flipset(x = x, clf = clf, action_set = action_set, solver= _SOLVER_TYPE_CPX).populate().items

In [5]:
items_cbc  = Flipset(x = x, clf = clf, action_set = action_set, solver= _SOLVER_TYPE_CBC).populate().items

obtained 10 items in 0.2 seconds


In [8]:
t_1 = [x['cost'] for x in items_cpx]
t_2 = [x['cost'] for x in items_cbc]
np.isclose(t_1, t_2).all()

In [22]:
t = fb_cbc.populate(enumeration_type="distinct_subsets")

obtained 10 items in 0.3 seconds


In [16]:
t = list(map(lambda x: x['actions'], t))

In [10]:
t

[array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 array([ 1.,  0.,  0.,  0., -6.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0., -1., -6.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 array([ 1.,  0.,  1.,  0., -6.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.

In [18]:
t[0].dot(t[1])

1004546.0

In [23]:
t_a = pd.DataFrame(np.array(list(map(lambda x: x['actions'], t))).astype(int))

In [24]:
t = t_a.pipe(lambda df: pd.DataFrame(~np.isclose(df, 0))).astype(int)

In [25]:
num_actions_on = t.sum(axis=1)

In [26]:
t

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1,0,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,1,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
6,1,0,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,1,0,1,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,1,0,1,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,1,0,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [58]:
num_actions_on

array([5, 4, 4, 4, 5, 4, 5, 5, 6, 5])

In [33]:
import itertools
from collections import defaultdict

In [36]:
overlap = defaultdict(dict)
for i,j in itertools.product(t.index, t.index):
    overlap[i][j] = t.loc[i].dot(t.loc[j])

In [67]:
pd.Series([1,2,3]).quantile(.8)

2.6

In [25]:
import pyomo.environ as pyo

In [26]:
pyo.ConstraintList()

In [4]:
feature_off_idxs = fb_cbc._mip_indices['action_off_names']
values_of_off_indices = [fb_cbc._mip.u[idx]() for idx in feature_off_idxs]
on_idx = np.isclose(values_of_off_indices, 0.0)
#
## array where con_val[i] = 1 if feature is off, -1 if feature is on.
con_vals = np.ones(len(feature_off_idxs), dtype = np.float_)
con_vals[on_idx] = -1.0

## one minus number of features that are off.
con_rhs = np.sum(~on_idx) - 1

In [5]:
feature_off_idxs = fb_cbc._mip_indices['action_off_names']
vars_of_off_indices = [fb_cbc._mip.u[idx] for idx in feature_off_idxs]

In [10]:
np.array(vars_of_off_indices).dot(con_vals)

In [12]:
action_series = pd.Series(cplex_solution['actions'], index=X.columns)

In [15]:
action_series

ForeignWorker                      0.0
Single                             0.0
Age                                0.0
LoanDuration                       0.0
LoanAmount                         0.0
LoanRateAsPercentOfIncome         -0.0
YearsAtCurrentHome                -0.0
NumberOfOtherLoansAtBank           0.0
NumberOfLiableIndividuals          0.0
HasTelephone                       0.0
CheckingAccountBalance_geq_0       0.0
CheckingAccountBalance_geq_200     1.0
SavingsAccountBalance_geq_100      0.0
SavingsAccountBalance_geq_500      1.0
MissedPayments                     0.0
NoCurrentLoan                      0.0
CriticalAccountOrLoansElsewhere    0.0
OtherLoansAtBank                   0.0
HasCoapplicant                     0.0
HasGuarantor                       1.0
OwnsHouse                          0.0
RentsHouse                         0.0
Unemployed                         0.0
YearsAtCurrentJob_lt_1             0.0
YearsAtCurrentJob_geq_4            0.0
JobClassIsSkilled        

In [11]:
fb_cplex._mip_indices['action_off_names']

['u[3][0]',
 'u[4][0]',
 'u[5][0]',
 'u[6][0]',
 'u[9][0]',
 'u[11][0]',
 'u[12][0]',
 'u[13][0]',
 'u[19][0]',
 'u[24][0]']

In [18]:
len(fb_cplex._mip_indices['coefficients'])

10

In [19]:
len(fb_cplex._mip_indices['action_off_names'])

10

In [20]:
fb_cplex._mip_indices['action_off_names']

['u[3][0]',
 'u[4][0]',
 'u[5][0]',
 'u[6][0]',
 'u[9][0]',
 'u[11][0]',
 'u[12][0]',
 'u[13][0]',
 'u[19][0]',
 'u[24][0]']

In [26]:
actions, percentiles = fb_cplex._action_set.feasible_grid(fb_cplex._x, return_actions=True, return_percentiles=True, return_immutable=False)

In [36]:
list((filter(lambda x: len(x[1]) >= 2, actions.items())))

[('LoanDuration',
  array([-42., -41., -40., -39., -38., -37., -36., -35., -34., -33., -32.,
         -31., -30., -29., -28., -27., -26., -25., -24., -23., -22., -21.,
         -20., -19., -18., -17., -16., -15., -14., -13., -12., -11., -10.,
          -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.])),
 ('LoanAmount',
  array([-5526., -5388., -5250., -5112., -4974., -4836., -4698., -4560.,
         -4422., -4284., -4146., -4008., -3870., -3732., -3594., -3456.,
         -3318., -3180., -3042., -2904., -2766., -2628., -2490., -2352.,
         -2214., -2076., -1938., -1800., -1662., -1524., -1386., -1248.,
         -1110.,  -972.,  -834.,  -696.,  -558.,  -420.,  -282.,  -144.,
            -6.,     0.])),
 ('LoanRateAsPercentOfIncome', array([-1.,  0.])),
 ('YearsAtCurrentHome', array([-1.,  0.])),
 ('HasTelephone', array([0., 1.])),
 ('CheckingAccountBalance_geq_200', array([0., 1.])),
 ('SavingsAccountBalance_geq_100', array([0., 1.])),
 ('SavingsAccountBalance_geq_500', arr

In [37]:
### if the action_off_index ==0, that means the feature is on.

In [ ]:
def remove_all_features(self):
    """
    removes feature combination from feasible region of MIP
    :return:
    """
    mip = self._mip
    names = self._mip_indices['action_off_names']
    values = np.array(mip.solution.get_values(names))
    on_idx = np.flatnonzero(np.isclose(values, 0.0))
    
    ### set a lower bound of 1
    mip.variables.set_lower_bounds([(names[j], 1.0) for j in on_idx])

In [41]:
u_names = fb_cplex._mip_indices['action_off_names']

In [43]:
mip = fb_cplex._mip
feature_off_idxs = fb_cplex._mip_indices['action_off_names']
u = np.array(mip.solution.get_values(feature_off_idxs))
## if the "off index" are off (i.e. = 0), that means the action is "on"
on_idx = np.isclose(u, 0.0)

In [45]:
n = len(feature_off_idxs)
con_vals = np.ones(n, dtype = np.float_)
con_vals[on_idx] = -1.0
con_rhs = n - 1 - np.sum(on_idx)

In [46]:
con_rhs

6

In [47]:
con_vals

array([ 1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1.])

In [49]:
np.sum(on_idx)

3

In [50]:
u

array([1., 1., 1., 1., 1., 0., 1., 0., 0., 1.])

In [51]:
n

10

In [55]:
t = pd.Series(data=con_vals, index=u_names)

In [76]:
t = t.to_frame('con vals')

In [83]:
t['u'] = u

In [84]:
t

,con vals,u
u[3][0],1.0,1.0
u[4][0],1.0,1.0
u[5][0],1.0,1.0
u[6][0],1.0,1.0
u[9][0],1.0,1.0
u[11][0],-1.0,0.0
u[12][0],1.0,1.0
u[13][0],-1.0,0.0
u[19][0],-1.0,0.0
u[24][0],1.0,1.0


In [87]:
t.apply(lambda x: x['con vals'] * x['u'], axis=1).sum()

7.0

In [91]:
t['con vals'].dot(t['u'])

7.0

In [73]:
'  +  '.join(map(lambda x: '%s * %s' %(x[1], x[0]), list(t.iteritems())))

'1.0 * u[3][0]  +  1.0 * u[4][0]  +  1.0 * u[5][0]  +  1.0 * u[6][0]  +  1.0 * u[9][0]  +  -1.0 * u[11][0]  +  1.0 * u[12][0]  +  -1.0 * u[13][0]  +  -1.0 * u[19][0]  +  1.0 * u[24][0]'

In [57]:
t.sum()

4.0

In [59]:
sum(on_idx)

3

In [63]:
n - 1 - 3

6

In [64]:
sum(~on_idx) - 1

6

In [65]:
t.sum()

4.0

In [105]:
fb_cbc.build_mip()

In [107]:
mip = fb_cbc._mip

In [110]:
fb_cbc._results = optimizer.solve(mip)

In [115]:
fb_cbc._results.solver.status

EnumValue(<pyutilib.enum.enum.Enum object at 0x00000265F46508D0>, 0, 'ok')

In [117]:
SolverStatus.ok

EnumValue(<pyutilib.enum.enum.Enum object at 0x00000265F46508D0>, 0, 'ok')

In [122]:
fb_cbc._results.solver

In [138]:
fb_cbc._results.solver.termination_condition == TerminationCondition.infeasible

False

In [140]:
fb_cbc.model